In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
gdrive_save_location = "/content/gdrive/My Drive/733 Project Files/training result"

In [ ]:
!ls "{gdrive_save_location}"

regular_small  test  tiny_full	tiny_full2  tiny_small	visual_test_custom


# Download YOLO code and weights

In [3]:
import sys
import torch
from google.colab import drive
import glob
print(f"Python version: {sys.version}, {sys.version_info} ")
print(f"Pytorch version: {torch.__version__} ")

Python version: 3.9.16 (main, Dec  7 2022, 01:11:51) 
[GCC 9.4.0], sys.version_info(major=3, minor=9, micro=16, releaselevel='final', serial=0) 
Pytorch version: 1.13.1+cu116 


In [4]:
%%capture

!# Download YOLOv7 code
!git clone https://github.com/WongKinYiu/yolov7
%cd yolov7
!ls

Cloning into 'yolov7'...
remote: Enumerating objects: 1139, done.
remote: Total 1139 (delta 0), reused 0 (delta 0), pack-reused 1139
Receiving objects: 100% (1139/1139), 70.31 MiB | 33.19 MiB/s, done.
Resolving deltas: 100% (514/514), done.
/content/yolov7
cfg	detect.py  hubconf.py  models	  requirements.txt  tools	  utils
data	export.py  inference   paper	  scripts	    train_aux.py
deploy	figure	   LICENSE.md  README.md  test.py	    train.py


In [ ]:
%%capture

!# Download trained weights
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-tiny.pt
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt

In [6]:
# download trained weights

!cp "{gdrive_save_location}/tiny_small/tiny_small/weights/best.pt" tiny_small.pt
!cp "{gdrive_save_location}/regular_small/weights/best.pt" regular_small.pt
!cp "{gdrive_save_location}/tiny_full/weights/best.pt" tiny_full.pt # not finished

In [ ]:
# get best mAP

!echo tiny_small mAP:    $(tail -n 1 "/content/gdrive/My Drive/733 Project Files/training result/tiny_small/tiny_small/results.txt" | awk -F" " ' {print $11} ')
!echo regular_small mAP: $(tail -n 1 "/content/gdrive/My Drive/733 Project Files/training result/regular_small/results.txt" | awk -F" " ' {print $11} ')
!echo tiny_full mAP:     $(tail -n 1 "/content/gdrive/My Drive/733 Project Files/training result/tiny_full/results.txt" | awk -F" " ' {print $11} ')
!echo
!echo tiny_small mAP@0.95:    $(tail -n 1 "/content/gdrive/My Drive/733 Project Files/training result/tiny_small/tiny_small/results.txt" | awk -F" " ' {print $12} ')
!echo regular_small mAP@0.95: $(tail -n 1 "/content/gdrive/My Drive/733 Project Files/training result/regular_small/results.txt" | awk -F" " ' {print $12} ')
!echo tiny_full mAP@0.95:     $(tail -n 1 "/content/gdrive/My Drive/733 Project Files/training result/tiny_full/results.txt" | awk -F" " ' {print $12} ')

# Download Custom Test Images

Details: https://learnopencv.com/fine-tuning-yolov7-on-custom-dataset/

In [15]:
%%capture

!gdown 1LduALiAiIoaN-6Gz-wyq-xI2pKyBi3ES
!unzip -o test_images.zip -d test_images

In [16]:
!mv test_images/test_images custom_test
!rm -r test_images

#Download udacity Data

In [ ]:
%%capture

# download udacity dataset
!gdown 1eA8A7Ol-MzAoZO1SzGvDHEkNimDcn_gK
!unzip -o udacity.zip -d udacity_test

In [9]:
# rename folder and move cfg files
!mv udacity_test/udacity udacity
!rm -r udacity_test

!mv udacity/udacity.yaml data/udacity.yaml
!mv udacity/yolov7-tiny-udacity.yaml cfg/training/yolov7-tiny-udacity.yaml
!mv udacity/yolov7-udacity.yaml cfg/training/yolov7-udacity.yaml

#Run YOLO test

In [ ]:
# test on tiny_small with full udacity
save_dir = "tiny_small" # result save dir
data_cfg = "data/udacity.yaml" # data.yaml
weight = 'tiny_small.pt' # weight name

!python test.py --device 0 --batch-size 16 \
--data {data_cfg} --img 640 --conf 0.001 --iou 0.65  \
--weights {weight} --name {save_dir}

Namespace(weights=['tiny_small.pt'], data='data/udacity.yaml', batch_size=16, img_size=640, conf_thres=0.001, iou_thres=0.65, task='val', device='0', single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project='runs/test', name='tiny_small', exist_ok=False, no_trace=False, v5_metric=False)
YOLOR 🚀 v0.1-122-g3b41c2c torch 1.13.1+cu116 CUDA:0 (Tesla T4, 15101.8125MB)

Fusing layers... 
IDetect.fuse
Model Summary: 208 layers, 6010302 parameters, 0 gradients
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

/usr/local/lib/python3.9/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
val: Scanning 'udacity/labels/valid.cache' images and labels... 3000 found, 0 mi

In [ ]:
# test on regular_small with full udacity
save_dir = "regular_small" # result save dir
data_cfg = "data/udacity.yaml" # data.yaml
weight = 'regular_small.pt' # weight name

!python test.py --device 0 --batch-size 16 \
--data {data_cfg} --img 640 --conf 0.001 --iou 0.65  \
--weights {weight} --name {save_dir}

_gdrive_save = f"{gdrive_save_location}/test/{save_dir}"
_result = f"runs/test/{save_dir}"
!cp -r "{_result}" "{_gdrive_save}"

Namespace(weights=['regular_small.pt'], data='data/udacity.yaml', batch_size=16, img_size=640, conf_thres=0.001, iou_thres=0.65, task='val', device='0', single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project='runs/test', name='regular_small', exist_ok=False, no_trace=False, v5_metric=False)
YOLOR 🚀 v0.1-122-g3b41c2c torch 1.13.1+cu116 CUDA:0 (Tesla T4, 15101.8125MB)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
Model Summary: 314 layers, 36487166 parameters, 6194944 gradients
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

/usr/local/lib/python3.9/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[att

In [10]:
# test on tiny_full with full udacity
save_dir = "tiny_full" # result save dir
data_cfg = "data/udacity.yaml" # data.yaml
weight = 'tiny_full.pt' # weight name

!python test.py --device 0 --batch-size 16 \
--data {data_cfg} --img 640 --conf 0.001 --iou 0.65  \
--weights {weight} --name {save_dir}

_gdrive_save = f"{gdrive_save_location}/test/{save_dir}"
_result = f"runs/test/{save_dir}"
!cp -r "{_result}" "{_gdrive_save}"

Namespace(weights=['tiny_full.pt'], data='data/udacity.yaml', batch_size=16, img_size=640, conf_thres=0.001, iou_thres=0.65, task='val', device='0', single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project='runs/test', name='tiny_full', exist_ok=False, no_trace=False, v5_metric=False)
YOLOR 🚀 v0.1-122-g3b41c2c torch 1.13.1+cu116 CUDA:0 (Tesla T4, 15101.8125MB)

Fusing layers... 
IDetect.fuse
Model Summary: 208 layers, 6010302 parameters, 0 gradients
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

/usr/local/lib/python3.9/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
val: Scanning 'udacity/labels/valid' images and labels... 321 found, 0 missing, 17

In [ ]:
# test on tiny_full with full udacity
save_dir = "yolov7" # result save dir
data_cfg = "data/udacity80.yaml" # data.yaml
weight = 'yolov7.pt' # weight name

!python test.py --device 0 --batch-size 16 \
--data {data_cfg} --img 640 --conf 0.001 --iou 0.65  \
--weights {weight} --name {save_dir}

_gdrive_save = f"{gdrive_save_location}/test/{save_dir}"
_result = f"runs/test/{save_dir}"
!cp -r "{_result}" "{_gdrive_save}"

#Run Custom Test

In [20]:
save_dir = "tiny_small"
weight = "tiny_small.pt"

!python detect.py --weights {weight} --conf 0.25 --img-size 640 --name {save_dir} --source custom_test/

save_dir = "regular_small"
weight = "regular_small.pt"

!python detect.py --weights {weight} --conf 0.25 --img-size 640 --name {save_dir} --source custom_test/

save_dir = "tiny_full"
weight = "tiny_full.pt"

!python detect.py --weights {weight} --conf 0.25 --img-size 640 --name {save_dir} --source custom_test/

save_dir = "original_tiny"
weight = "yolov7-tiny.pt"

!python detect.py --weights {weight} --conf 0.25 --img-size 640 --name {save_dir} --source custom_test/

save_dir = "original_regular"
weight = "yolov7.pt"

!python detect.py --weights {weight} --conf 0.25 --img-size 640 --name {save_dir} --source custom_test/

Namespace(weights=['tiny_small.pt'], source='custom_test/', img_size=640, conf_thres=0.25, iou_thres=0.45, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='tiny_small', exist_ok=False, no_trace=False)
YOLOR 🚀 v0.1-122-g3b41c2c torch 1.13.1+cu116 CUDA:0 (Tesla T4, 15101.8125MB)

Fusing layers... 
IDetect.fuse
Model Summary: 208 layers, 6010302 parameters, 0 gradients
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

/usr/local/lib/python3.9/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
3 cars, Done. (6.6ms) Inference, (1.5ms) NMS
 The image with the result is saved in: runs/detect/tiny_small2/test1.jpg

In [19]:
!cp -r "runs/detect" "{gdrive_save_location}/visual_test_custom" 